# Laboratorium 4 - rekomendacje dla portali informacyjnych

## Przygotowanie

 * pobierz i wypakuj dataset: https://mind201910small.blob.core.windows.net/release/MINDsmall_train.zip
   * więcej możesz poczytać tutaj: https://learn.microsoft.com/en-us/azure/open-datasets/dataset-microsoft-news
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab4`
 * zainstaluj potrzebne biblioteki:
 `pip install nltk sklearn`

## Część 1. - przygotowanie danych

In [71]:
# importujemy wszystkie potrzebne pakiety

import codecs
from collections import (
    defaultdict,
)  # mozesz uzyc zamiast zwyklego slownika, rozwaz wplyw na czas obliczen
import math
import re
from string import punctuation

import nltk

nltk.download("stopwords")
nltk.download("punkt")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from nltk.stem import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer

# mozesz uzyc do obliczania najbardziej podobnych tekstow zamiast liczenia "na piechote"
# ale pamietaj o dostosowaniu formatu danych
from sklearn.neighbors import NearestNeighbors

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/przemek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/przemek/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [72]:
# definiujemy potrzebne zmienne

PATH = "./MINDsmall_train"
STOPWORDS = set(stopwords.words("english"))

In [73]:
# wczytujemy metadane artykułów


def parse_news_entry(entry):
    news_id, category, subcategory, title, abstract = entry.split("\t")[:5]
    return {
        "news_id": news_id,
        "category": category,
        "subcategory": subcategory,
        "title": title,
        "abstract": abstract,
    }


def get_news_metadata():
    with codecs.open(f"{PATH}/news.tsv", "r", "UTF-8") as f:
        raw = [x for x in f.read().split("\n") if x]
        parsed_entries = [parse_news_entry(entry) for entry in raw]
        return {x["news_id"]: x for x in parsed_entries}


news = get_news_metadata()
news_ids = sorted(list(news.keys()))
news_indices = {x[1]: x[0] for x in enumerate(news_ids)}
print(len(news))

51282


## Część 2. - TF-IDF

In [74]:
# normalizujemy teksty na potrzeby dalszego przetwarzania


def preprocess_text(text):
    # usuwamy znaki interpunkcyjne
    regex = re.compile("[%s]" % re.escape(punctuation))
    text = regex.sub("", text)

    # usuwamy wszystkie liczby
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", text)

    # podmieniamy wszystkie wielkie litery
    text = text.lower()

    # dzielimy na tokeny
    word_tokens = word_tokenize(text)
    # usuwamy stopwords
    text = " ".join([w for w in word_tokens if w not in STOPWORDS])
    # zamieniamy wszystkie ciagi bialych znakow na pojedyncze spacje
    _RE_COMBINE_WHITESPACE = re.compile(r"\s+")
    text = _RE_COMBINE_WHITESPACE.sub(" ", text).strip()

    return text.split()


def stem_texts(corpus):
    stemmer = PorterStemmer()
    return [[stemmer.stem(word) for word in preprocess_text(text)] for text in corpus]


texts = [news[news_id]["abstract"] for news_id in news_ids]
stemmed_texts = stem_texts(texts)

In [75]:
# porownajmy teksty przed i po przetworzeniu

print(texts[2] + "\n")
print(" ".join(stemmed_texts[2]))

"I think we have a really good team, and a team that can really do some special, good things because that group is very close in there." - Brian Schmetzer

think realli good team team realli special good thing group close brian schmetzer


In [76]:
# tworzymy liste wszystkich slow w korpusie


def get_all_words_sorted(corpus):
    wordlist = set()
    for text in corpus:
        for word in text:
            wordlist.add(word)

    return sorted(list(wordlist))


wordlist = get_all_words_sorted(stemmed_texts)
word_indices = {x[1]: x[0] for x in enumerate(wordlist)}
print(len(wordlist))

44023


In [77]:
# obliczamy liczbe tekstow, w ktorych wystapilo kazde ze slow
# pamietaj, ze jesli slowo wystapilo w danym tekscie wielokrotnie, to liczymy je tylko raz


def get_document_frequencies(corpus, wordlist):
    doc_freq = {word: 0 for word in wordlist}

    for text in corpus:
        text_wordlist = set(text)
        for word in text_wordlist:
            doc_freq[word] += 1

    return doc_freq


document_frequency = get_document_frequencies(stemmed_texts, wordlist)

In [78]:
# obliczamy liczbe wystapien kazdego slowa w kazdym tekscie


def get_term_frequencies(corpus, news_indices):
    term_freq = {news_id: defaultdict(lambda: 0) for news_id in news_indices.keys()}

    for idx, (text, news_id) in enumerate(zip(corpus, news_indices.keys())):
        for word in text:
            term_freq[news_id][word] += 1

    return term_freq


term_frequency = get_term_frequencies(stemmed_texts, news_indices)

In [79]:
# sprawdzmy wyniki

term_frequency[news_ids[2]]

defaultdict(<function __main__.get_term_frequencies.<locals>.<dictcomp>.<lambda>()>,
            {'think': 1,
             'realli': 2,
             'good': 2,
             'team': 2,
             'special': 1,
             'thing': 1,
             'group': 1,
             'close': 1,
             'brian': 1,
             'schmetzer': 1})

In [80]:
# obliczamy metryke tf_idf


def calculate_tf_idf(term_frequency, document_frequency, corpus_size):
    tf_idf = {news_id: defaultdict(lambda: 0) for news_id in term_frequency.keys()}

    for news_id in term_frequency.keys():
        for word in term_frequency[news_id].keys():
            tf_idf[news_id][word] = term_frequency[news_id][word] * math.log(
                corpus_size / document_frequency[word] + 1
            )

    return tf_idf


tf_idf = calculate_tf_idf(term_frequency, document_frequency, len(news_ids))

In [81]:
# sprawdzmy wyniki

tf_idf[news_ids[2]]

defaultdict(<function __main__.calculate_tf_idf.<locals>.<dictcomp>.<lambda>()>,
            {'think': 4.373151488960738,
             'realli': 9.72280058058018,
             'good': 8.12019624698702,
             'team': 6.2481518012360855,
             'special': 4.773647302723159,
             'thing': 4.127940511650616,
             'group': 4.275685924628469,
             'close': 3.9439845389396853,
             'brian': 5.6283402794893425,
             'schmetzer': 9.458878728310335})

## Część 3. - Podobieństwo tekstów

In [82]:
# obliczmy odleglosc miedzy dwoma artykulami
# przetestuj rozne metryki odleglosci i wybierz najlepsza


def calculate_distance(tf_idf, id1, id2):
    # cosine similarity
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for word in tf_idf[id1].keys():
        sum1 += tf_idf[id1][word] * tf_idf[id2][word]
        sum2 += tf_idf[id1][word] ** 2
    for word in tf_idf[id2].keys():
        sum3 += tf_idf[id2][word] ** 2

    if sum2 == 0 or sum3 == 0:
        return 0

    return sum1 / (math.sqrt(sum2) * math.sqrt(sum3))


calculate_distance(tf_idf, news_ids[2], news_ids[1])

0.02529386978912101

In [83]:
tf_idf["N10062"]

defaultdict(<function __main__.calculate_tf_idf.<locals>.<dictcomp>.<lambda>()>,
            {})

In [85]:
# wyznaczmy k najpodobniejszych tekstow do danego
# pamietaj o odpowiedniej kolejnosci sortowania w zaleznosci od wykorzystanej metryki
# pamietaj, zeby wsrod podobnych tekstow nie bylo danego


def get_k_most_similar_news(tf_idf, n_id, k):
    similarities = []
    for news_id in tf_idf.keys():
        if news_id != n_id:
            similarities.append((news_id, calculate_distance(tf_idf, n_id, news_id)))

    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    return list(map(lambda x: x[0], similarities[0:k]))


def print_k_most_similar_news(tf_idf, n_id, k, corpus, news_indices):
    similar = get_k_most_similar_news(tf_idf, n_id, k)
    print(f"id: {n_id}, text: {corpus[news_indices[n_id]]}")
    print(f"\n{k} most similar:")
    for s_id in similar:
        print(f"\nid: {s_id}, text: {corpus[news_indices[s_id]]}")


print_k_most_similar_news(tf_idf, news_ids[42337], 5, texts, news_indices)

id: N58544, text: A MAN claims he has created a car that might solve the world's traffic congestion problems. Rick Woodbury from Spokane, Washington USA, is the president, founder and sole employee of 'Commuter Cars.' The carmaker's flagship model is the 2005 super slim two-seater Tango T600, a high-performance electric car that preceded Tesla. Rick told BTV: "I started this company 21 years ago   it was based on an idea that I came up with in 1982." He was inspired by the shocking traffic congestion he had to face on a daily basis. "I used to drive a Porsche from Beverly Hills to Hermosa Beach every day and the traffic was horrendous," explained Rick. What really made Rick think about a solution was the fact that in most of the cars he would see in his commute were occupied by lone drivers. "I noticed that everybody around me was a single occupant in a car, taking up the whole lane," Rick said. Living and working in Los Angeles also helped inspire Rick's unique creation. "I thought, e